# Assignment 13

## Problem 7.10
In the main text we derived the trapezoid and Simpson’s rules from Lagrange
interpolation; we now continue along that path, for the case of Simpson’s 3/8 rule.
For $q = 4$ analytically derive the elementary-interval (a) weights from Eq. (7.27), and (b) absolute error, by integrating Eq. (6.38). Compare with table 7.1.

**Solution:**
In Simpson's 3/8 rule we divide the integration window into 3 panels to approximate the function to a cubic polynomial.
The Lagrange basis polynomials are:

$$
L_0(x) = \frac{(x-x_1)(x-x_2)(x-x_3)}{(x_0-x_1)(x_0-x_2)(x_0-x_3)}\\
L_1(x) = \frac{(x-x_0)(x-x_2)(x-x_3)}{(x_1-x_0)(x_1-x_2)(x_1-x_3)}\\
L_2(x) = \frac{(x-x_0)(x-x_1)(x-x_3)}{(x_2-x_0)(x_2-x_1)(x_2-x_3)}\\
L_3(x) = \frac{(x-x_0)(x-x_1)(x-x_2)}{(x_3-x_0)(x_3-x_1)(x_3-x_2)}\\
$$

To derive the weights for Simpson's 3/8 rule, we integrate the Lagrange interpolating polynomial over the interval $[x_0, x_3]$. If the interval width is $h$, the points will be $x_0$, $x_1 = x_0 + h$, $x_2 = x_0 + 2h$, and $x_3 = x_0 + 3h$.

Let us make a substitution
$$
u=\frac{x-x_0}{h}
$$
Now, integrating each Lagrange basis polynomial from $x_0$ to $x_3$, we get after simplification,

$$
\int_{x_0}^{x_3} L_0(x)\,dx = \frac{3h}{8}\\
\int_{x_0}^{x_3} L_1(x)\,dx = \frac{9h}{8}\\
\int_{x_0}^{x_3} L_2(x)\,dx = \frac{9h}{8}\\
\int_{x_0}^{x_3} L_3(x)\,dx = \frac{3h}{8}\\
$$

Therefore the three panel version of Simpson's rule can be derived from these weights as,

$$
\boxed{\int_{x_0}^{x_3} f(x)\,dx \approx \frac{3h}{8} \left[f(x_0)+3f(x_1)+3f(x_2)+f(x_3)\right]}
$$

**Error Analysis:**

Consider a function $f(x)$ and its expansion using a Taylor series around a point $x_0$,
$$
f(x) = f(x_0) + f^{(1)}(x_0)(x-x_0) + \frac{1}{2!} f^{(2)}(x_0)(x-x_0)^2 + \frac{1}{3!}f^{(3)}(x_0)(x-x_0)^3 + \frac{1}{2!}f^{(4)}(\xi)(x-x_0)^4
$$
where $\xi$ is a point between $x$ and $x_0$. Now, applying the Taylor series expansion to each point $x_0, x_1, x_2, x_3$ and substituting into Simpson's 3/8 rule formula, we find that every term the third derivative of $f(x)$ cancel out. An hence, the error term will be,

$$
\boxed{\epsilon = -\frac{3}{80} h^5 f^{(4)}(\xi)}
$$

# Problem 7.12
Starting from the elementary-interval expressions for Simpson’s 3/8 rule and for
Boole’s rule given in table 7.1, implement the composite versions of these approaches.
Then, generalize the derivation of section 7.3.1 to cover Boole’s rule and then modify
our code in `adaptive.py` so that it also works for this case.

$$
I = S^{(1)} + E^{(1)} = S(a,b) + E(a,b), \\
I = S^{(2)} + E^{(2)} = S(a,c) + S(c,d)+ S(d,b) + E^{(2)} \\
\left|E^{(2)}\right| \le \frac{3}{80}\left(\frac{h/2}{2}\right)^5 |f^{(5)}(\xi_1)| + \frac{3}{80}\left(\frac{h/2}{2}\right)^5 |f^{(5)}(\xi_2)| + \frac{3}{80}\left(\frac{h/2}{2}\right)^5 |f^{(5)}(\xi_3)|
$$
Assuming the derivative term stays constant,
$$
\left|E^{(2)}\right| \le 3\cdot\frac{3}{80}\left(\frac{1}{2^{10}}\right) h^5C \\
\left|E^{(1)}\right| \le \frac{3}{80}\left(\frac{1}{2^5}\right) h^5C \\
E^{(1)} - E^{(2)}= 16E^{(2)} - E^{(2)} = 15E^{(2)}
$$

In [7]:
import numpy as np

def f(x):
  return 1/np.sqrt(x**2 + 1)
  # return np.cos(x)

def simpson_3by8(f,a,b,n):
  h = (b-a)/(n-1)
  xs = a + np.arange(n)*h
  cs = 3*np.ones(n)
  cs[3:n-2:3] = 2
  cs[0] = cs[-1] = 1
  contribs = cs*f(xs)
  return (3*h/8)*np.sum(contribs)

def boole(f,a,b,n):
  h = (b-a)/(n-1)
  xs = a + np.arange(n)*h
  cs = np.ones(n)
  cs[1:n:2] = 32
  cs[2:n-1:4] = 12
  cs[4:n-3:4] = 14
  cs[0] = cs[-1] = 7
  contribs = cs*f(xs)
  return (2*h/45)*np.sum(contribs)

def simpson(f,a,b,n):
  h = (b-a)/(n-1)
  xs = a + np.arange(n)*h
  cs = 2*np.ones(n)
  cs[1::2] = 4; cs[0] = 1; cs[-1] = 1
  contribs = cs*f(xs)
  return (h/3)*np.sum(contribs)

def adaptive(f,a,b,integration_fn,kmax = 20,tol = 1.e-6):
  denom = 15
  n = 3

  val = integration_fn(f,a,b,n)
  for k in range(kmax):
    nprime = 2*n-1
    valprime = integration_fn(f,a,b,nprime)
    err = abs(valprime-val)/denom
    err /= abs(valprime)
    print(nprime, valprime, err)
    if err<tol:
      break
    n, val = nprime, valprime
  else:
    valprime = None
  return valprime

ans = np.log(1 + np.sqrt(2))
print('Answer')
print(ans); print("")
print('Using Simpson\'s 1/3 rule')
print(adaptive(f, 0., 1., simpson)); print("")
print('Using Simpson\'s 3/8 rule')
print(adaptive(f, 0., 1., simpson_3by8)); print("")
print('Using Boole\'s rule')
print(adaptive(f, 0., 1, boole)); print("")

Answer
0.8813735870195429

Using Simpson's 1/3 rule
5 0.8813775969806421 4.349298327553859e-05
9 0.8813739323106564 2.771937351709414e-07
0.8813739323106564

Using Simpson's 3/8 rule
5 0.9094514863708398 0.0063227613019875865
9 0.8696265391892474 0.003053030650660781
17 0.8872518127449023 0.0013243345577491597
33 0.8785681780246057 0.0006589232296743729
65 0.8827763734911004 0.00031780003731126673
129 0.880680353890626 0.00015866669378317567
257 0.8817202048924555 7.862290070852982e-05
513 0.8812007849131767 3.9296377410741356e-05
1025 0.8814599880932723 1.9604079867262107e-05
2049 0.881330418109527 9.801090191447111e-06
4097 0.8813951714748725 4.897792911447449e-06
8193 0.881362796767773 2.4488369767228436e-06
16385 0.881378982145433 1.2242465490927396e-06
32769 0.8813708895800793 6.121195552214502e-07
0.8813708895800793

Using Boole's rule
5 0.8814159307217272 0.0015256608460151812
9 0.8813736879993241 3.195218477566981e-06
17 0.8813735877394049 7.583611206443883e-09
0.88137358773940